In [ ]:
try:
    # --- Mount gdrive to /content/drive/My Drive/
    from google.colab import drive
    drive.mount('/content/drive')
    
except: pass

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# --- Set data directory
MOUNT_ROOT = '/content/drive/My Drive'
DATASET_ROOT = '/content/drive/MyDrive/Minecraft Segmentation Data Set'

In [ ]:
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl
# VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION

In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import  Dataset,DataLoader
from torchvision import transforms, utils
import cv2
import glob
from torch import nn
import time
import copy
import csv
from tqdm import tqdm
from collections import Counter

# # imports the torch_xla package
# import torch_xla
# import torch_xla.core.xla_model as xm

In [ ]:
import subprocess
def get_gpu_memory_map():
    """Get the current gpu usage.

    Returns
    -------
    usage: dict
        Keys are device ids as integers.
        Values are memory usage as integers in MB.
    """
    result = subprocess.check_output(
        [
            'nvidia-smi', '--query-gpu=memory.used',
            '--format=csv,nounits,noheader'
        ], encoding='utf-8')
    # Convert lines into a dictionary
    gpu_memory = [int(x) for x in result.strip().split('\n')]
    gpu_memory_map = dict(zip(range(len(gpu_memory)), gpu_memory))
    return gpu_memory_map


In [ ]:
MODELS_DIR_PATH = MOUNT_ROOT+"/Models"
def save_model(epoch, model, optimizer, loss):
  torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, MODELS_DIR_PATH + "/model_epoch_" + '{0}'.format(str(epoch).zfill(3)) + ".pt")

In [ ]:
from os import listdir, mkdir
from os.path import isfile, join, exists
import re
import glob

def extract_epoc(string):
    r = re.compile(r'(\d+)')
    return int(r.findall(string)[0])


def get_latest_checkpoint(model, optimizer):
    # if exists(MODELS_DIR_PATH + "/model_epoch_" + '{0}'.format(str(epoch-1).zfill(3)) + ".pt"):

    # only_files = [f for f in listdir(MODELS_DIR_PATH) if isfile(join(MODELS_DIR_PATH, f))]
    # sorted_files = sorted(onlyfiles ,key=lambda x: extract_epoch(x))
    # latest_file = sorted_files[-1] 

    latest_file = glob.glob(MODELS_DIR_PATH + "/model_epoch_*.pt")[0]
    checkpoint = torch.load(latest_file)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print("get_latest_checkpoint model address={}".format(hex(id(model))))
    print("Loaded successfully the latest checkpoint: epoch={} loss={}".format(epoch, loss))
    return epoch + 1


In [ ]:
from shutil import copyfile, rmtree
from math import ceil 

IMAGE_TOTAL = 13333
PARTITION_SIZE = 999
PARTITION_TOTAL = ceil(13333/PARTITION_SIZE)

def partition_images(video_dirs, mask_dirs, partition_total):
    
    mode=0o666 #read-write only
    for p in range(partition_total):
      video_p_dir = f'video_{p+1}'
      mask_p_dir = f'mask_p{p+1}'

      if exists(join(DATASET_ROOT, video_p_dir)):
        rmtree(join(DATASET_ROOT, video_p_dir))
        rmtree(join(DATASET_ROOT, mask_p_dir))

      mkdir(join(DATASET_ROOT, video_p_dir), mode=0o666)
      mkdir(join(DATASET_ROOT, mask_p_dir), mode=0o666)

      for file_num in range(p*PARTITION_SIZE, min((p+1)*(PARTITION_SIZE),IMAGE_TOTAL)):
        for i in range(len(video_dirs)):
          if isfile(join(DATASET_ROOT, video_dirs[i], f'{file_num}.png')):
            copyfile(join(DATASET_ROOT, video_dirs[i], f'{file_num}.png'), join(DATASET_ROOT, video_p_dir, f'{file_num}.png'))
            copyfile(join(DATASET_ROOT, mask_dirs[i], f'{file_num}.png'), join(DATASET_ROOT, mask_p_dir, f'{file_num}.png'))
            break
          
# partition_images(['video1', 'video2'],['mask1', 'mask2'], PARTITION_TOTAL)

In [ ]:
torch.cuda.empty_cache()
class MinecraftSegmentationDataset(Dataset):
    """Segmentation Dataset"""
 
    def __init__(self, root_dir, imageFolder, maskFolder, transform=None, seed=None, fraction=None, subset=None, imagecolormode='rgb', maskcolormode='grayscale'):
        """
        Args:
            root_dir (string): Directory with all the images and should have the following structure.
            root
            --Images
            -----Img 1
            -----Img N
            --Mask
            -----Mask 1
            -----Mask N
            imageFolder (string) = 'Images' : Name of the folder which contains the Images.
            maskFolder (string)  = 'Masks : Name of the folder which contains the Masks.
            transform (callable, optional): Optional transform to be applied on a sample.
            seed: Specify a seed for the train and test split
            fraction: A float value from 0 to 1 which specifies the validation split fraction
            subset: 'Train' or 'Test' to select the appropriate set.
            imagecolormode: 'rgb' or 'grayscale'
            maskcolormode: 'rgb' or 'grayscale'
        """
        self.color_dict = {'rgb': 1, 'grayscale': 0}
        assert(imagecolormode in ['rgb', 'grayscale'])
        assert(maskcolormode in ['rgb', 'grayscale'])
 
        self.imagecolorflag = self.color_dict[imagecolormode]
        self.maskcolorflag = self.color_dict[maskcolormode]
        self.root_dir = root_dir
        self.transform = transform
        if not fraction:
            self.image_names = sorted(
                glob.glob(os.path.join(self.root_dir, imageFolder, '*')))
            self.mask_names = sorted(
                glob.glob(os.path.join(self.root_dir, maskFolder, '*')))
        else:
            assert(subset in ['Train', 'Test'])
            self.fraction = fraction
            self.image_list = np.array(sorted(glob.glob(os.path.join(self.root_dir, imageFolder, '*'))))
            
            self.mask_list = np.array(
                sorted(glob.glob(os.path.join(self.root_dir, maskFolder, '*'))))
            if seed:
                np.random.seed(seed)
            indices = np.arange(len(self.image_list))
            np.random.shuffle(indices)
            self.image_list = self.image_list[indices]
            self.mask_list = self.mask_list[indices]
            if subset == 'Train':
                self.image_names = self.image_list[:int(
                    np.ceil(len(self.image_list)*(1-self.fraction)))]
                self.mask_names = self.mask_list[:int(
                    np.ceil(len(self.mask_list)*(1-self.fraction)))]
            else:
                self.image_names = self.image_list[int(
                    np.ceil(len(self.image_list)*(1-self.fraction))):]
                self.mask_names = self.mask_list[int(
                    np.ceil(len(self.mask_list)*(1-self.fraction))):]
 
    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        msk_name = self.mask_names[idx]
        if self.imagecolorflag:
            if cv2.imread(img_name, self.imagecolorflag) is None:
                print(img_name)
            image = cv2.imread(img_name, self.imagecolorflag).transpose(2, 0, 1)
        else:
            image = cv2.imread(img_name, self.imagecolorflag)
                                  
        if self.maskcolorflag:
            mask = cv2.imread(msk_name, self.maskcolorflag).transpose(2, 0, 1)
        else:
            mask = cv2.imread(msk_name, self.maskcolorflag)
        sample = {'image': image, 'mask': mask}
 
        if self.transform:
            sample = self.transform(sample)
 
        return sample
        
class Resize(object):
    """Resize image and/or masks."""

    def __init__(self, imageresize, maskresize):
        self.imageresize = imageresize
        self.maskresize = maskresize

    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        if len(image.shape) == 3:
            image = image.transpose(1, 2, 0)
        if len(mask.shape) == 3:
            mask = mask.transpose(1, 2, 0)
        mask = cv2.resize(mask, self.maskresize, cv2.INTER_AREA)
        image = cv2.resize(image, self.imageresize, cv2.INTER_AREA)
        if len(image.shape) == 3:
            image = image.transpose(2, 0, 1)
        if len(mask.shape) == 3:
            mask = mask.transpose(2, 0, 1)

        return {'image': image,
                'mask': mask}


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample, maskresize=None, imageresize=None):
        image, mask = sample['image'], sample['mask']
        if len(mask.shape) == 2:
            mask = mask.reshape((1,)+mask.shape)
        if len(image.shape) == 2:
            image = image.reshape((1,)+image.shape)
        return {'image': torch.from_numpy(image),
                'mask': torch.from_numpy(mask)}


class Normalize(object):
    '''Normalize image'''

    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        return {'image': image.type(torch.FloatTensor)/255,
                'mask': mask.type(torch.int64)}

def get_dataloader(data_dir, imageFolder='video', maskFolder='mask', fraction=0.2, batch_size=4):
    """
        Create training and testing dataloaders from a single folder.
    """
    data_transforms = {
        'Train': transforms.Compose([ToTensor(), Normalize()]),
        'Test': transforms.Compose([ToTensor(), Normalize()]),
    }

    image_datasets = {x: MinecraftSegmentationDataset(data_dir, imageFolder=imageFolder, maskFolder=maskFolder, seed=100, fraction=fraction, subset=x, transform=data_transforms[x])
                      for x in ['Train', 'Test']}
    dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size,
                                 shuffle=True)
                   for x in ['Train', 'Test']}
    return dataloaders

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import jaccard_score, f1_score
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from torchvision.models.segmentation import  deeplabv3_resnet101
class FocalLoss(nn.Module):
    def __init__(self, alpha=.25, gamma=2, size_average=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.size_average = size_average

    def forward(self, inputs, targets):
       
        ce_loss = F.cross_entropy(
            inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * ce_loss
        if self.size_average:
            return focal_loss.mean()
        else:
            return focal_loss.sum()

def custom_DeepLabv3(out_channel=132):
    model =  deeplabv3_resnet101(pretrained=True, progress=True)
    model.classifier = DeepLabHead(2048, out_channel)
    #Set the model in training mode
    model.train()
    return model


# print(get_gpu_memory_map())

In [ ]:
def train_model(model, criterion, dataloaders, optimizer, metrics, bpath, num_epochs=3, start_from_latest_checkpoint=False):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    # Use gpu if available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Initialize the log file for training and testing loss and metrics
    fieldnames = ['epoch', 'Train_loss', 'Test_loss'] + \
        [f'Train_{m}' for m in metrics.keys()] + \
        [f'Test_{m}' for m in metrics.keys()]

    
    start_epoch = 1
    start_loss = None
    log_open_option = 'w'

    if start_from_latest_checkpoint:
        start_epoch = get_latest_checkpoint(model, optimizer)
        print("Starting at epoch={}".format(start_epoch))

        # to append instead of truncating log.csv
        log_open_option = 'a+'


    with open(os.path.join(bpath, 'log.csv'), log_open_option, newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not start_from_latest_checkpoint:
            writer.writeheader()


    for epoch in range(start_epoch, num_epochs+1):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)
        # Each epoch has a training and validation phase
        # Initialize batch summary
        batchsummary = {a: [0] for a in fieldnames}
 
        for phase in ['Train', 'Test']:
            if phase == 'Train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
 
            # Iterate over data.
            for sample in tqdm(iter(dataloaders[phase])):
                inputs = sample['image'].to(device)
                masks = sample['mask'].to(device).squeeze(1)

                # zero the parameter gradients
                optimizer.zero_grad()
 
                # track history if only in train
                with torch.set_grad_enabled(phase == 'Train'):
                    outputs = model(inputs)
                    outputs['out'] = outputs['out']
                    outputs['out']=outputs['out'].double()

                    loss = criterion(outputs['out'], masks)
                    
                    outputs['out'] = outputs['out'].argmax(1)
                
                    y_pred = outputs['out'].data.cpu().numpy()
                   

                    y_pred = y_pred.ravel()
                    y_true = masks.data.cpu().numpy()
                  
                    y_true = y_true.ravel()
                   
                    for name, metric in metrics.items():
                        if name == 'f1_score':
                            # Use a classification threshold of 0.1
                            
                            batchsummary[f'{phase}_{name}'].append(metric(y_true > 0, y_pred > 0.1,average='micro'))
                        else:
                            
                            batchsummary[f'{phase}_{name}'].append(metric(y_true.astype('uint8'), y_pred,average='micro'))
 
                    # backward + optimize only if in training phase
                    if phase == 'Train':
                        loss.backward()
                        optimizer.step()
            batchsummary['epoch'] = epoch
            epoch_loss = loss
            batchsummary[f'{phase}_loss'] = epoch_loss.item()
            print('{} Loss: {:.4f}'.format(
                phase, loss))
        for field in fieldnames[3:]:
            batchsummary[field] = np.mean(batchsummary[field])
        print(batchsummary)
        with open(os.path.join(bpath, 'log.csv'), 'a', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(batchsummary)
            # deep copy the model
            if phase == 'Test' and loss < best_loss:
                best_loss = loss
                best_model_wts = copy.deepcopy(model.state_dict())
        # save
        save_model(epoch, model, optimizer, loss)
        # delete the previous model
        if exists(MODELS_DIR_PATH + "/model_epoch_" + '{0}'.format(str(epoch-1).zfill(3)) + ".pt"):
            os.remove(MODELS_DIR_PATH + "/model_epoch_" + '{0}'.format(str(epoch-1).zfill(3)) + ".pt")

 
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Lowest Loss: {:4f}'.format(best_loss))
 
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
    

In [ ]:
def save_dataloaders(dataloaders):
  dataloaders['Train']
  dataloaders['']

In [ ]:
torch.cuda.empty_cache()
# print(get_gpu_memory_map())
root = r'/content/drive/MyDrive/Minecraft Segmentation Data Set'
imageFolder = 'video'
maskFolder = 'mask'
model = custom_DeepLabv3(132)
metrics = {'IoU': jaccard_score, 'f1_score': f1_score}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_func = FocalLoss()
MODELS_PATH = join(MOUNT_ROOT, "CopyModels")

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth


In [ ]:
def save_dataloaders(dataloaders, dir_path):
  torch.save(dataloaders['Train'], join(dir_path, 'train.pth'))
  torch.save(dataloaders['Test'], join(dir_path, 'test.pth'))

In [ ]:
def load_dataloaders(dir_path):
    return {"Train": torch.load(join(dir_path, 'train.pth')), "Test": torch.load(join(dir_path, 'test.pth'))}

In [ ]:
start_from_latest_checkpoint = False

In [ ]:
# for i in range(2,3):
#   dataloaders = None
#   if start_from_latest_checkpoint:
#     dataloaders = load_dataloaders(MODELS_PATH)
#   else:
#     dataloaders = get_dataloader(root,imageFolder=f'video{i}',maskFolder=f'mask{i}', batch_size=3)
#     save_dataloaders(dataloaders, MODELS_PATH)
#   print("before calling train_model model address={}".format(hex(id(model))))
#   model = train_model(model, loss_func, dataloaders, optimizer, metrics, MODELS_PATH, 11, start_from_latest_checkpoint)

In [ ]:
dataloaders = None
if start_from_latest_checkpoint:
  dataloaders = load_dataloaders(MODELS_PATH)
else:
  dataloaders = get_dataloader(root,imageFolder=f'small_video',maskFolder=f'small_mask', batch_size=3)
  save_dataloaders(dataloaders, MODELS_PATH)
print("before calling train_model model address={}".format(hex(id(model))))
model = train_model(model, loss_func, dataloaders, optimizer, metrics, MODELS_PATH, 500, start_from_latest_checkpoint)

before calling train_model model address=0x7fdb178aab70


  0%|          | 0/533 [00:00<?, ?it/s]

Epoch 1/500
----------


  0%|          | 0/133 [00:00<?, ?it/s]

Train Loss: 0.0919


...

100%|██████████| 133/133 [01:53<00:00,  1.17it/s]


Test Loss: 0.0325
{'epoch': 46, 'Train_loss': 0.026819872937566616, 'Test_loss': 0.03249600513423786, 'Train_IoU': 0.8721751193490349, 'Train_f1_score': 0.99812734082397, 'Test_IoU': 0.8675053310685175, 'Test_f1_score': 0.9925373134328358}


  0%|          | 0/533 [00:00<?, ?it/s]

Epoch 47/500
----------


 24%|██▍       | 127/533 [03:19<10:31,  1.56s/it]